## Transfer Data Configuration: Cofigure necessary components to perform a Data Transfer in the next notebook


### Description

[Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) 
Connect to your Azure Machine Learning service Workspace. Note that if you are using a Compute Instance the config file is already there for the Workspace you're within, otherwise you will need to create it.

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

### Description

[Datastore.register_azure_sql_database](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore%28class%29?view=azure-ml-py#register-azure-sql-database-workspace--datastore-name--server-name--database-name--tenant-id-none--client-id-none--client-secret-none--resource-url-none--authority-url-none--endpoint-none--overwrite-false--username-none--password-none----kwargs-) This classe allows you to register a number of Azure services to be used in your Pipelines including,

- Azure Blob Container
- Azure File Share
- Azure Data Lake
- Azure Data Lake Gen2
- Azure SQL Database
- Azure Database for PostgreSQL
- Databricks File System
- Azure Database for MySQL

Once you run this code, check your Azure Machine Learning Studio -> Data Sources to see the Azure SQL Database registered.

**Note** Create an application under Azure Active Directory in order to have a client_id, and a client_secret.
1. Azure Active Directory -> App Registrations -> + New Registeration
2. Click into the App -> Certs and Secrets -> New client Secret
3. Connect to the Azure SQL Database and add this application as a login to the database and provide it appropriate permissions. In this case, the application was called 'mlonazure',

CREATE USER [mlonazure] FROM  EXTERNAL PROVIDER  WITH DEFAULT_SCHEMA=[dbo]
GO
sp_addrolemember db_datareader, mlonazure
sp_addrolemember db_datawriter, mlonazure


In [ ]:
from azureml.core import Datastore

datastore_name = 'azuresqldb_datastore'
server_name = ''
database_name = ''

client_id = ''
client_secret = ''
tenant_id = ''

mlonazuresql_ds = Datastore.register_azure_sql_database(
        workspace=mlonazure_ws,
        datastore_name=datastore_name,
        server_name=server_name,
        database_name=database_name,
        tenant_id=tenant_id,
        client_id=client_id,
        client_secret=client_secret)

### Description:

[DataFactoryCompute.attach_configuration](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.datafactory.datafactorycompute?view=azure-ml-py#attach-configuration-resource-group-none--factory-name-none--resource-id-none-) Provides configuration for an existing Azrue Data Factory

[ComputeTarget.attach](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py#attach-workspace--name--attach-configuration-) Attaches the existing Azure Data Factory as an "Attached Compute" under "Compute" in Studio (Review your Azure Machine Learning studio after you run this code). This allows for this ADF to be used later in an DataTransferStep

In [ ]:
from azureml.core.compute import ComputeTarget, DataFactoryCompute 

rg = 'mlonazure-rg'
adf='mlonazureadf'
adfcompute = 'mlonazure-adf'

adfconfig = DataFactoryCompute.attach_configuration(resource_group=rg, factory_name=adf, resource_id=None)
adfcompute = ComputeTarget.attach(workspace=mlonazure_ws, name=adfcompute, attach_configuration=adfconfig)
adfcompute.wait_for_completion()
